# Chart generations Notebook

## Import Libraries

In [1]:
import pandas as pd
import plotly.graph_objs as go
import plotly.offline as offline
import os
import numpy as np
from plotly.offline import plot

In [2]:
offline.init_notebook_mode(connected=True)

## Bubble Chart for Emissions Intensity


In [3]:
company_id = 1

# Step 1: Set up the data sources

In [4]:
XLSX_PATH = os.path.join ('../django_project/static/django_project', 'data', 'sp100_data.xlsx')
COLS_TO_USE = {
    'companies':['company_id', 'company_name'],
    'GHG19':['company_id', 'gross_total_scope1', 'gross_scope2_calc'],
    'grouping':['company_id', 'Sector1'],
    'financials':['company_id', 'Revenue']
}

# Step 2: Merge the tables

In [5]:
all_df = []
for sheetname, cols in COLS_TO_USE.items():
    all_df.append(pd.read_excel(
        XLSX_PATH, 
        sheet_name = sheetname,
        engine = 'openpyxl', 
        usecols = cols
        ))

In [6]:
merged_df = all_df[0]
for i in range(len(COLS_TO_USE) - 1):
    merged_df = pd.merge(
        left = merged_df,
        right = all_df[i+1],
        how="left",
        on="company_id"
    )

In [7]:
merged_df.head()

,company_id,company_name,gross_total_scope1,gross_scope2_calc,Sector1,Revenue
0,1,3M,4050000,1320000.0,Industrials,3.213600e+10
1,2,Abbott Laboratories,533000,439000.0,Healthcare,3.190400e+10
2,3,AbbVie Inc.,314421,249777.0,Healthcare,3.326600e+10
3,4,Accenture,18923,214680.0,Technology,4.320000e+10
4,5,Adobe Inc.,11816,43526.0,Technology,1.117100e+10


# Step 3: Filter the sector

In [8]:
a =  merged_df[merged_df['company_id']==company_id]['Sector1']
if len(a) > 0:
    sector = a.iloc[0]
merged_df_sector = merged_df.loc[merged_df['Sector1']==sector]

In [9]:
merged_df_sector.head()

,company_id,company_name,gross_total_scope1,gross_scope2_calc,Sector1,Revenue
0,1,3M,4050000,1320000.0,Industrials,3.213600e+10
19,20,Boeing,613000,733000.0,Industrials,7.655900e+10
23,24,Caterpillar Inc.,NF,0.0,Industrials,5.380000e+10
39,40,Emerson Electric,NF,0.0,Industrials,1.837200e+10
43,44,FedEx,15406173,995988.0,Industrials,6.969300e+10


# Step 4: Calculate Scope 1 + Scope 2

In [10]:
scope1 = pd.to_numeric(merged_df_sector['gross_total_scope1'], errors = 'coerce')
scope2 = pd.to_numeric(merged_df_sector['gross_scope2_calc'], errors = 'coerce')
merged_df_sector = merged_df_sector.assign(scope1_plus_scope2= scope1 + scope2)

# Step 5: Remove companies without valid data

In [11]:
merged_df_sector = merged_df_sector.dropna()

# Step 6 (optional): Calculate GHG Intensity

In [12]:
merged_df_sector.assign(intensity=merged_df_sector['scope1_plus_scope2'] / merged_df_sector['Revenue'])

,company_id,company_name,gross_total_scope1,gross_scope2_calc,Sector1,Revenue,scope1_plus_scope2,intensity
0,1,3M,4050000,1320000.000,Industrials,3.213600e+10,5.370000e+06,0.000167
19,20,Boeing,613000,733000.000,Industrials,7.655900e+10,1.346000e+06,0.000018
43,44,FedEx,15406173,995988.000,Industrials,6.969300e+10,1.640216e+07,0.000235
45,46,General Dynamics,317081,445119.000,Industrials,3.935000e+10,7.622000e+05,0.000019
46,47,General Electric,997410,1387615.018,Industrials,9.521400e+10,2.385025e+06,0.000025
51,52,Honeywell Inc.,1090649,933484.000,Industrials,3.670900e+10,2.024133e+06,0.000055
58,59,Lockheed Martin Corp.,305362,466073.000,Industrials,5.981000e+10,7.714350e+05,0.000013
91,92,Union Pacific Corp,10066595,314075.000,Industrials,2.170000e+10,1.038067e+07,0.000478
93,94,United Parcel Service,14223000,731000.000,Industrials,7.409400e+10,1.495400e+07,0.000202


# Step 7: Prepare the chart by fetching specific data series

In [13]:
corp_x = merged_df_sector[merged_df_sector['company_id']==company_id]['scope1_plus_scope2'].iloc[0]
corp_y = merged_df_sector[merged_df_sector['company_id']==company_id]['Revenue'].iloc[0]
corp_name = merged_df_sector[merged_df_sector['company_id']==company_id]['company_name'].iloc[0]
x0 = merged_df_sector['scope1_plus_scope2']
y0 = merged_df_sector['Revenue']
index_corp = merged_df_sector.loc[merged_df_sector['company_id'] == company_id].index
merged_df_sector.drop(index_corp , inplace=True)
x_median_x = [x0.min(), x0.max()]
y_median_x = [y0.median(), y0.median()]
x_median_y = [x0.median(), x0.median()]
y_median_y = [y0.min(), y0.max()]



# Step 7: Produce the chart

In [14]:
layout = go.Layout (
    title = '<b>GHG Emissions vs. Revenue</b><br>Sector: ' + sector,
    title_x = 0.5,
    titlefont = dict(
        family = 'Arial',
        size = 16),
    plot_bgcolor = 'antiquewhite',
    xaxis =  dict(
        autorange = "reversed",
        type = 'log',
        title = '<i>high</i>----------<b>Scope1+2 Emissions</b>----------<i>low</i>'),
    yaxis = dict(
        type = 'log',
        title = 'Revenue'),
        )

In [15]:
trace1 = go.Scatter(x=x_median_y, 
                    y=y_median_y, 
                    showlegend = False, 
                    name='Median y',
                    mode = "lines",
                    line = dict(color='gray', 
                                width=2, 
                                dash='dash'))

trace2 = go.Scatter(x=x_median_x, 
                         y=y_median_x, 
                         showlegend = False, 
                         name='Median x', 
                         mode = "lines",
                         line = {'color':'gray', 
                                 'width':2, 
                                 'dash':'dash'}
                        )

trace3 = go.Scatter(
    x=merged_df_sector['scope1_plus_scope2'], 
    y=merged_df_sector['Revenue'],
    showlegend = False,
    name="benchmark", 
    text=merged_df_sector['company_name'],
    mode = 'markers+text',
    line = {'color':'black'}, 
    marker = {
        'color': '#488f31',
        'opacity': 0.4,
        'size':12
    },
    textposition='top center'
    )
trace4 = go.Scatter(
    x=[corp_x], 
    y=[corp_y],
    showlegend = False,
    name=corp_name, 
    text= corp_name,
    textfont=dict(
        size=12,
        color="#de425b"),
    mode = 'markers+text',
    line = {'color':'#de425b'}, 
    marker = {
        'color': '#de425b',
        'opacity': 1,
        'size':20
    },
    textposition='top center'
    )

fig = go.Figure(data = [trace1, trace2, trace3, trace4], 
                layout = layout)
                
fig.update_layout(
    autosize=False,
    width=500,
    height=500,
    margin=dict(
        l=50,
        r=50,
        b=50,
        t=80,
        pad=4
    ),
    paper_bgcolor="#bad0af",
    plot_bgcolor = '#f1f1f1',
)
config = {'displaylogo': False}


In [16]:
offline.iplot(fig, config = config)

In [17]:
name_fig = "intensity_idx{}".format(company_id)
fig.write_image("../django_project/static/django_project/images/charts/{}.svg".format(name_fig), scale=3, height = 300)
plot(fig, config=config, filename = '../django_project/templates/django_project/corporates/charts/html_exports/{}.html'.format(name_fig), auto_open=False)

'../django_project/templates/django_project/corporates/charts/html_exports/intensity_idx1.html'